<a href="https://colab.research.google.com/github/biranchi2018/My-NLP-Examples/blob/master/3.BERT%20-%20Masked_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 665kB 9.0MB/s 
     |████████████████████████████████| 890kB 24.2MB/s 
     |████████████████████████████████| 1.1MB 31kB/s 
     |████████████████████████████████| 3.8MB 11.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=de36cc32e1485206ec71b8bead4395089cbb586f7a1e213635a2803bbcf83f52
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


BERT - Masked Language Model
=============================

https://becominghuman.ai/practical-ai-using-pretrained-bert-to-generate-grammar-and-vocabulary-multiple-choice-questions-d92e4fbeeeaa


# Example 1

In [0]:
import re
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [0]:
import time
start = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
end = time.time()
print ("Time Elapsed to load BERT ",end-start)


Time Elapsed to load BERT  15.730551719665527


In [0]:
def get_predicted_words(text):
    text = "[CLS] " + text.replace("____", "[MASK]") + " [SEP]"
    # text= '[CLS] Tom has fully [MASK] from his illness. [SEP]'
    
    tokenized_text = tokenizer.tokenize(text)
    
    #print("tokenized sentence: ",tokenized_text,"\n")
    masked_index = tokenized_text.index('[MASK]')
    print(f'masked_index : {masked_index}')
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

	  # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

	  # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

	  # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)[0]
	
	  # Get 30 choices for the masked(blank) word
    #print(f'predictions : {predictions}') 
    k = 30
    predicted_index, predicted_index_values = torch.topk(predictions[0, masked_index], k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_index_values.tolist())
    filtered_tokens_to_remove_punctuation = []
    
    # Remove any predictions that contain punctuation etc as they are not relevant to us.
    for token in predicted_tokens:
        if re.match("^[a-zA-Z0-9_]*$", token):
            filtered_tokens_to_remove_punctuation.append(token)
        
    return filtered_tokens_to_remove_punctuation

In [0]:
sentence = "They all look tiny ____ they are so far away from the Earth."
print("original sentence: ",sentence,"\n")

original sentence:  They all look tiny ____ they are so far away from the Earth. 



In [0]:
predicted_words = get_predicted_words(sentence)
print()
print("predicted choices:\n", predicted_words)

masked_index : 5
predictions : tensor([[[ -6.5230,  -6.5132,  -6.5371,  ...,  -5.8959,  -5.7181,  -3.9671],
         [-13.1075, -13.0090, -12.6393,  ..., -10.3502, -11.1232, -10.2471],
         [-14.8110, -15.1335, -14.6437,  ..., -13.3993, -13.3356, -13.7805],
         ...,
         [-11.3714, -11.3986, -11.2618,  ...,  -8.9173,  -9.7927,  -8.4863],
         [-14.4480, -14.0974, -14.4010,  ..., -12.0537, -11.8662,  -9.4477],
         [-19.7119, -19.8447, -19.6689,  ..., -17.6284, -17.9101, -11.4495]]])

predicted choices:
 ['because', 'and', 'but', 'as', 'since', 'like', 'when', 'considering', 'for', 'except', 'though', 'that', 'yet', 'although', 'while', 'now', 'if', 'where', 'here', 'even', 'so', 'after', 'from', 'given']


# Example 2

In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = '[CLS] I want to [MASK] the car because it is cheap . [SEP]'
tokenized_text = tokenizer.tokenize(text)
tokenized_text

['[CLS]',
 'i',
 'want',
 'to',
 '[MASK]',
 'the',
 'car',
 'because',
 'it',
 'is',
 'cheap',
 '.',
 '[SEP]']

In [0]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
indexed_tokens

[101, 1045, 2215, 2000, 103, 1996, 2482, 2138, 2009, 2003, 10036, 1012, 102]

In [0]:
# Create the segments tensors.
segments_ids = [0] * len(tokenized_text)
segments_ids

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [0]:
# Predict all tokens
with torch.no_grad():
	predictions = model(tokens_tensor, segments_tensors)[0]
	print(f"predictions: {predictions}")

predictions: tensor([[[ -6.7150,  -6.6846,  -6.6816,  ...,  -6.1106,  -5.9049,  -4.0863],
         [-11.7569, -11.4073, -11.5342,  ..., -10.7097,  -9.0829,  -8.8668],
         [-11.0520, -10.7464, -11.3275,  ...,  -9.3687,  -8.6992, -10.8701],
         ...,
         [ -6.4314,  -6.5334,  -6.3702,  ...,  -7.1500,  -7.4019,  -4.8901],
         [-12.4604, -11.9011, -12.6061,  ...,  -9.6319, -10.1272,  -8.3597],
         [-13.8967, -13.8270, -14.0311,  ..., -13.3388, -11.5974, -10.7325]]])


In [0]:
masked_index = tokenized_text.index('[MASK]')
print(f"masked_index : {masked_index}")

predicted_index = torch.argmax(predictions[0, masked_index]).item()
print(f"predicted_index : {predicted_index}")

predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(f"predicted_token : {predicted_token}")


masked_index : 4
predicted_index : 4965
predicted_token : buy


# Example 3


AI plays Mad Libs and the results are terrifying 

https://towardsdatascience.com/a-i-plays-mad-libs-and-the-results-are-terrifying-78fa44e7f04e


In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [7]:
text = (
    "Today, every student has a computer small enough to fit into "
    "his _. He can solve any math problem by simply pushing the "
    "computer's little _. Computers can add, multiply, divide, and "
    "_. They can also _ better than a human. Some computers are "
    "_. Others have an _ screen that shows all kinds of _ and _ "
    "figures. "
)
text

"Today, every student has a computer small enough to fit into his _. He can solve any math problem by simply pushing the computer's little _. Computers can add, multiply, divide, and _. They can also _ better than a human. Some computers are _. Others have an _ screen that shows all kinds of _ and _ figures. "

In [8]:
# Load pre-trained model with masked language model head
bert_version = 'bert-large-uncased'
# bert_version = 'bert-base-uncased'

model = BertForMaskedLM.from_pretrained(bert_version)

In [9]:
# Preprocess text
tokenizer = BertTokenizer.from_pretrained(bert_version)
tokenized_text = tokenizer.tokenize(text)
mask_positions = []
for i in range(len(tokenized_text)):
    if tokenized_text[i] == '_':
        tokenized_text[i] = '[MASK]'
        mask_positions.append(i)

mask_positions

[13, 29, 41, 46, 55, 60, 67, 69]

In [10]:
# Predict missing words from left to right
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementw

In [11]:
tokenized_text

['today',
 ',',
 'every',
 'student',
 'has',
 'a',
 'computer',
 'small',
 'enough',
 'to',
 'fit',
 'into',
 'his',
 '[MASK]',
 '.',
 'he',
 'can',
 'solve',
 'any',
 'math',
 'problem',
 'by',
 'simply',
 'pushing',
 'the',
 'computer',
 "'",
 's',
 'little',
 '[MASK]',
 '.',
 'computers',
 'can',
 'add',
 ',',
 'multi',
 '##ply',
 ',',
 'divide',
 ',',
 'and',
 '[MASK]',
 '.',
 'they',
 'can',
 'also',
 '[MASK]',
 'better',
 'than',
 'a',
 'human',
 '.',
 'some',
 'computers',
 'are',
 '[MASK]',
 '.',
 'others',
 'have',
 'an',
 '[MASK]',
 'screen',
 'that',
 'shows',
 'all',
 'kinds',
 'of',
 '[MASK]',
 'and',
 '[MASK]',
 'figures',
 '.']

In [0]:
for mask_pos in mask_positions:
    # Convert tokens to vocab indices
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([token_ids])
    
    # Call BERT to predict token at this position
    #predictions = model(tokens_tensor)[0, mask_pos]
    predictions = model(tokens_tensor)[0]
    
    predicted_index = torch.argmax(predictions).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    
    # Update text
    tokenized_text[mask_pos] = predicted_token


In [13]:
tokenized_text

['today',
 ',',
 'every',
 'student',
 'has',
 'a',
 'computer',
 'small',
 'enough',
 'to',
 'fit',
 'into',
 'his',
 '[UNK]',
 '.',
 'he',
 'can',
 'solve',
 'any',
 'math',
 'problem',
 'by',
 'simply',
 'pushing',
 'the',
 'computer',
 "'",
 's',
 'little',
 '[UNK]',
 '.',
 'computers',
 'can',
 'add',
 ',',
 'multi',
 '##ply',
 ',',
 'divide',
 ',',
 'and',
 '[UNK]',
 '.',
 'they',
 'can',
 'also',
 '[UNK]',
 'better',
 'than',
 'a',
 'human',
 '.',
 'some',
 'computers',
 'are',
 '[UNK]',
 '.',
 'others',
 'have',
 'an',
 '[UNK]',
 'screen',
 'that',
 'shows',
 'all',
 'kinds',
 'of',
 '[UNK]',
 'and',
 '[UNK]',
 'figures',
 '.']

In [14]:
for mask_pos in mask_positions:
    tokenized_text[mask_pos] = "_" + tokenized_text[mask_pos] + "_"


print(f"original text :\n", text)

result = ' '.join(tokenized_text).replace(' ##', '')

print(f"result text :\n", result)


original text :
 Today, every student has a computer small enough to fit into his _. He can solve any math problem by simply pushing the computer's little _. Computers can add, multiply, divide, and _. They can also _ better than a human. Some computers are _. Others have an _ screen that shows all kinds of _ and _ figures. 
result text :
 today , every student has a computer small enough to fit into his _[UNK]_ . he can solve any math problem by simply pushing the computer ' s little _[UNK]_ . computers can add , multiply , divide , and _[UNK]_ . they can also _[UNK]_ better than a human . some computers are _[UNK]_ . others have an _[UNK]_ screen that shows all kinds of _[UNK]_ and _[UNK]_ figures .


# Example 4

In [3]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 8.0MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM


In [5]:

text = (
    "Today, every student has a computer small enough to fit into "
    "his _. He can solve any math problem by simply pushing the "
    "computer's little _. Computers can add, multiply, divide, and "
    "_. They can also _ better than a human. Some computers are "
    "_. Others have an _ screen that shows all kinds of _ and _ "
    "figures. "
)

# Load pre-trained model with masked language model head
bert_version = 'bert-large-uncased'
model = BertForMaskedLM.from_pretrained(bert_version)

# Preprocess text
tokenizer = BertTokenizer.from_pretrained(bert_version)
tokenized_text = tokenizer.tokenize(text)
mask_positions = []
for i in range(len(tokenized_text)):
    if tokenized_text[i] == '_':
        tokenized_text[i] = '[MASK]'
        mask_positions.append(i)

# Predict missing words from left to right
model.eval()
for mask_pos in mask_positions:
    # Convert tokens to vocab indices
    token_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([token_ids])
    # Call BERT to predict token at this position
    predictions = model(tokens_tensor)[0, mask_pos]
    predicted_index = torch.argmax(predictions).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    # Update text
    tokenized_text[mask_pos] = predicted_token

for mask_pos in mask_positions:
    tokenized_text[mask_pos] = "_" + tokenized_text[mask_pos] + "_"
print(' '.join(tokenized_text).replace(' ##', ''))


100%|██████████| 231508/231508 [00:00<00:00, 897952.99B/s]


today , every student has a computer small enough to fit into his _bed_ . he can solve any math problem by simply pushing the computer ' s little _button_ . computers can add , multiply , divide , and _kill_ . they can also _be_ better than a human . some computers are _black_ . others have an _open_ screen that shows all kinds of _clothes_ and _even_ figures .
